In [1]:
# !pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

In [2]:
# !pip install --upgrade accelerat
# !pip uninstall -y transformers accelerat
# !pip install transformers accelerate

In [3]:
import os
import sys
os.chdir('../')

In [4]:
pwd

'c:\\Users\\Viral Sherathiya\\Desktop\\NLP Data\\Text_Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass (frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size:int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps:int

In [6]:
from textSummarization.constants import *
from textSummarization.utils.common import *
from textSummarization import logger,CustomException



class ConfigurationManager:

    def __init__(
            self,
            config_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        try:
            config = self.config.model_trainer
            params = self.params.TrainingArguments
            create_directories([config.root_dir])

            model_trainer_config = ModelTrainerConfig(
                root_dir= config.root_dir,
                data_path=config.data_path,
                model_ckpt=config.model_ckpt,
                num_train_epochs=params.num_train_epochs,
                per_device_train_batch_size=params.per_device_train_batch_size,
                gradient_accumulation_steps=params.gradient_accumulation_steps,
                warmup_steps=params.warmup_steps,
                logging_steps=params.logging_steps,
                eval_steps=params.eval_steps,
                evaluation_strategy=params.evaluation_strategy,
                save_steps=params.save_steps,
                weight_decay=params.weight_decay
            )

            return model_trainer_config
        except Exception as e:
                logger.info(f"Error Occurred at {CustomException(e,sys)}")
                raise CustomException(e, sys)

In [7]:
from transformers import (TrainingArguments, Trainer,DataCollatorForSeq2Seq,AutoModelForSeq2SeqLM, AutoTokenizer)
from datasets import load_dataset, load_from_disk
import torch
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        device = 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer= ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-08-13 19:24:28,751] || common.py || 37 || TextSummarizerLogger || read_yaml() || 37 || INFO || yaml file : config\config.yaml loaded successfully
[2023-08-13 19:24:28,758] || common.py || 37 || TextSummarizerLogger || read_yaml() || 37 || INFO || yaml file : config\params.yaml loaded successfully
[2023-08-13 19:24:28,760] || common.py || 69 || TextSummarizerLogger || create_directories() || 69 || INFO || Directory artifacts created successfully
[2023-08-13 19:24:28,762] || common.py || 69 || TextSummarizerLogger || create_directories() || 69 || INFO || Directory artifacts/model_trainer created successfully


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Viral Sherathiya\Desktop\NLP Data\Text_Summarizer\venv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/51 [00:00<?, ?it/s]

You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 376.00 MiB (GPU 0; 4.00 GiB total capacity; 9.45 GiB already allocated; 0 bytes free; 9.79 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF